This notebook is currently in progress

# GEE Aster Preprocessing

## Step 1: Create a google earth engine account

Follow this [link](https://code.earthengine.google.com/register) to create a GEE account

## Step 2: Installation


In [1]:
%pip install git+https://github.com/Mining-for-the-Future/ASTER_preprocessing.git@function_disaggregation

  Cloning https://github.com/Mining-for-the-Future/ASTER_preprocessing.git (to revision function_disaggregation) to c:\users\eweav\appdata\local\temp\pip-req-build-8w2gxuf4
  Resolved https://github.com/Mining-for-the-Future/ASTER_preprocessing.git to commit 0076d7a35d1bb1ef59d369640f066b1e8002bbcb
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/Mining-for-the-Future/ASTER_preprocessing.git 'C:\Users\eweav\AppData\Local\Temp\pip-req-build-8w2gxuf4'
  Running command git checkout -b function_disaggregation --track origin/function_disaggregation
  branch 'function_disaggregation' set up to track 'origin/function_disaggregation'.
  Switched to a new branch 'function_disaggregation'


## Step 3: Importing Libraries

In [ ]:
%pip install geemap

importing these modules should trigger the google earth engine authentication process.

Note: in VS Code, the input field for the google earth engine token is at the top of the window, not underneath the cell.

In [2]:
from ASTER_preprocessing.preprocessing import aster_collection_preprocessing, aster_image_preprocessing
from ASTER_preprocessing.vis_params import vis_params_image


Successfully saved authorization token.


In [3]:
import geemap

## Step 4: Preprocess ASTER image collection using a drawn polygon

In [4]:
# Create an interactive map
Map = geemap.Map(center = [-24.269444, -69.070556], zoom = 10)
Map

Map(center=[-24.269444, -69.070556], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

In [5]:
# Get the last drawn feature
geometry = Map.user_roi

In [6]:
collection = aster_collection_preprocessing(geometry)

## Step 5: Visualize the image collection as a time series

In [7]:
first_image = collection.first()
vis_params = vis_params_image(first_image, bands = ['B04', 'B3N', 'B02'])

map2 = geemap.Map()
map2.centerObject(first_image)
# map2.addLayer(first_image, vis_params)
map2.add_time_slider(collection, vis_params = vis_params, time_interval = 1)
map2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Step 5A: automatically retrieve the image with the highest geographic coverage

In [39]:
def get_best_image(image_coll):
    return image_coll.sort('geom_coverage', ascending = False).first()

In [40]:
best_image = get_best_image(collection)
map3 = geemap.Map()
map3.centerObject(best_image)
map3.addLayer(best_image, vis_params_image(best_image, bands = ['B04', 'B06', 'B08']), 'best image')
map3

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Step 6 Collect images from multiple regions

In [8]:
Map = geemap.Map(center = [-24.269444, -69.070556], zoom = 10)
Map

Map(center=[-24.269444, -69.070556], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

In [41]:
fcoll = Map.user_rois

In [42]:
fcoll

In [44]:
def get_geom_from_feat(feat):
    return feat.geometry()

In [43]:
def coll_preprocess_from_feat(feat, bands = ['B01', 'B02', 'B3N', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14'], masks = [], cloudcover = 25):
    geom = get_geom_from_feat(feat)
    return aster_collection_preprocessing(geom, bands, masks, cloudcover)

In [45]:
def get_best_image_from_feat(feat, bands = ['B01', 'B02', 'B3N', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14'], masks = [], cloudcover = 25):
    return get_best_image(coll_preprocess_from_feat(feat, bands, masks, cloudcover))

In [46]:
f_image_coll = fcoll.map(lambda x: get_best_image_from_feat(x))

In [47]:
f_image_coll

In [54]:
image_collection = ee_i.ImageCollection(f_image_coll)

In [56]:
vis_params = vis_params_image(image_collection.first(), bands = ['B04', 'B06', 'B08'])

In [57]:
map4 = geemap.Map()
map4.centerObject(f_image_coll)
map4.addLayer(image_collection, vis_params = vis_params)
map4

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [59]:
type(image_collection)

ee.imagecollection.ImageCollection

In [60]:
ee_i.Algorithms.ObjectType(image_collection)

In [14]:
f_image_coll = fcoll.map(lambda x: aster_collection_preprocessing(get_geom_from_feat(x)))

In [15]:
from ASTER_preprocessing.preprocessing import ee_i

In [16]:
ee_i.Algorithms.ObjectType(f_image_coll)

In [19]:
f_image_coll

In [29]:
ee_i.ImageCollection(f_image_coll.first()).size()

In [27]:
map = geemap.Map(center = [-24.269444, -69.070556], zoom = 10)
map.addLayer(ee_i.ImageCollection(f_image_coll.first()).first())
map

Map(center=[-24.269444, -69.070556], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…